In [9]:
import os  # 운영 체제와 상호작용하기 위한 모듈
import shutil  # 파일 및 디렉토리 작업을 위한 모듈
import random  # 랜덤 작업을 위한 모듈
import scipy  # 과학 및 기술 계산을 위한 모듈
import numpy as np  # 수치 연산을 위한 Python 라이브러리
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리
from PIL import Image  # 이미지 처리 및 파일 입출력을 위한 라이브러리
from keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터 생성기 유틸리티
from keras.models import Model  # Keras의 함수형 API 모델
from keras.layers import Activation, Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D, Add, GlobalAveragePooling2D  # 신경망의 레이어 구성 요소
from keras.optimizers import Adam, SGD  # 최적화 알고리즘
from keras.callbacks import EarlyStopping, ModelCheckpoint  # 학습 중 콜백 함수

def generators(train_dir, val_dir, size=16):
    """학습 및 검증 데이터를 생성하는 함수"""
    train_datagen = ImageDataGenerator(rescale=1/255,  # 이미지의 픽셀 값을 0-1 범위로 정규화
                                       rotation_range=180,  # 최대 180도 회전
                                       width_shift_range=0.2,  # 가로 방향 이동
                                       height_shift_range=0.2,  # 세로 방향 이동
                                       shear_range=0.2,  # 시어 변환
                                       zoom_range=0.2,  # 줌 인/줌 아웃
                                       horizontal_flip=True,  # 가로 방향 뒤집기
                                       vertical_flip=True,  # 세로 방향 뒤집기
                                       fill_mode='nearest')  # 변환 중 생기는 빈 공간을 주변의 유사한 픽셀 값으로 채우기

    val_datagen = ImageDataGenerator(rescale=1/255)  # 검증 데이터 정규화

    # 학습 데이터 생성
    train_generator = train_datagen.flow_from_directory(train_dir,  # 학습 데이터가 위치한 디렉토리
                                                        target_size=(299, 299),  # 입력 이미지 크기
                                                        batch_size=size,  # 배치 크기
                                                        class_mode='categorical',  # 다중 클래스 분류
                                                        shuffle=True)  # 데이터를 섞어서 배치 생성

    # 검증 데이터 생성
    val_generator = val_datagen.flow_from_directory(val_dir,  # 검증 데이터가 위치한 디렉토리
                                                    target_size=(299, 299),  # 입력 이미지 크기
                                                    batch_size=size,  # 배치 크기
                                                    class_mode='categorical')  # 다중 클래스 분류

    return train_generator, val_generator  # 학습 및 검증 데이터 생성기 반환

def block(input_tensor, filters, kernel_size=3, strides=1, padding='same', use_bias=False):
    """기본 블록을 정의하는 함수"""
    x = SeparableConv2D(filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias)(input_tensor)  # Depthwise Separable Convolution 레이어
    x = BatchNormalization()(x)  # 배치 정규화 레이어
    x = Activation('relu')(x)  # ReLU 활성화 함수 적용
    return x

def Custom(input_shape=(299, 299, 3), num_classes=100):
    """커스텀 CNN 모델을 정의하는 함수"""
    img_input = Input(shape=input_shape)  # 입력 텐서 정의

    # Entry flow
    x = Conv2D(32, 3, strides=2, padding='same', use_bias=False)(img_input)  # 첫 번째 Conv2D 레이어
    x = BatchNormalization()(x)  # 배치 정규화 레이어
    x = Activation('relu')(x)  # ReLU 활성화 함수 적용

    x = Conv2D(64, 3, padding='same', use_bias=False)(x)  # 두 번째 Conv2D 레이어
    x = BatchNormalization()(x)  # 배치 정규화 레이어
    x = Activation('relu')(x)  # ReLU 활성화 함수 적용

    residual = Conv2D(128, 1, strides=2, padding='same', use_bias=False)(x)  # 잔차 연결을 위한 Conv2D 레이어
    residual = BatchNormalization()(residual)  # 배치 정규화 레이어

    x = block(x, 128)  # 첫 번째 블록
    x = block(x, 128)  # 두 번째 블록
    x = MaxPooling2D(3, strides=2, padding='same')(x)  # MaxPooling 레이어
    x = Add()([x, residual])  # 잔차 연결

    residual = Conv2D(256, 1, strides=2, padding='same', use_bias=False)(x)  # 잔차 연결을 위한 Conv2D 레이어
    residual = BatchNormalization()(residual)  # 배치 정규화 레이어

    x = block(x, 256)  # 세 번째 블록
    x = block(x, 256)  # 네 번째 블록
    x = MaxPooling2D(3, strides=2, padding='same')(x)  # MaxPooling 레이어
    x = Add()([x, residual])  # 잔차 연결

    residual = Conv2D(728, 1, strides=2, padding='same', use_bias=False)(x)  # 잔차 연결을 위한 Conv2D 레이어
    residual = BatchNormalization()(residual)  # 배치 정규화 레이어

    x = block(x, 728)  # 다섯 번째 블록
    x = block(x, 728)  # 여섯 번째 블록
    x = MaxPooling2D(3, strides=2, padding='same')(x)  # MaxPooling 레이어
    x = Add()([x, residual])  # 잔차 연결

    # Middle flow
    for _ in range(8):
        residual = x
        x = block(x, 728)  # 중간 블록 반복
        x = block(x, 728)  # 중간 블록 반복
        x = block(x, 728)  # 중간 블록 반복
        x = Add()([x, residual])  # 잔차 연결

    # Exit flow
    residual = Conv2D(1024, 1, strides=2, padding='same', use_bias=False)(x)  # 잔차 연결을 위한 Conv2D 레이어
    residual = BatchNormalization()(residual)  # 배치 정규화 레이어

    x = block(x, 728)  # 첫 번째 Exit 블록
    x = block(x, 1024)  # 두 번째 Exit 블록
    x = MaxPooling2D(3, strides=2, padding='same')(x)  # MaxPooling 레이어
    x = Add()([x, residual])  # 잔차 연결

    x = block(x, 1536, kernel_size=3, strides=1)  # Conv2D 레이어
    x = block(x, 2048, kernel_size=3, strides=1)  # Conv2D 레이어

    x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링 레이어 적용
    output = Dense(num_classes, activation='softmax')(x)  # 출력 레이어 (Softmax 활성화 함수 사용)

    model = Model(img_input, output)  # 모델 정의
    model.compile(optimizer=SGD(learning_rate=.01, momentum=.9, decay=.01), loss='categorical_crossentropy', metrics=['accuracy'])  # 모델 컴파일
    model.summary()  # 모델 요약 출력

    return model

# 커스텀 모델 생성
model = Custom()

# 모델 저장 경로 및 콜백 설정
model_path = 'your_path'  # 최적의 모델을 저장할 파일 경로
CP = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)  # 최적의 모델을 저장하는 콜백
ES = EarlyStopping(monitor='val_loss', patience=10)  # 조기 종료를 위한 콜백, 10 에포크 동안 개선이 없으면 학습 중단

# 모델 학습
history = model.fit(train_generator, epochs=200, validation_data=val_generator, callbacks=[CP, ES])  # 학습 수행

# 학습 결과 시각화
plt.plot(history.history['loss'])  # 학습 손실 그래프
plt.plot(history.history['accuracy'])  # 학습 정확도 그래프
plt.plot(history.history['val_loss'])  # 검증 손실 그래프
plt.plot(history.history['val_accuracy'])  # 검증 정확도 그래프
plt.title('Custom V3 Model')  # 그래프 제목
plt.ylabel('loss')  # y축 라벨
plt.xlabel('epoch')  # x축 라벨
plt.legend(['loss', 'accuracy', 'val_loss', 'val_accuracy'], loc='upper right')  # 범례
plt.show()  # 그래프 출력

# 최적의 검증 정확도와 손실 출력
print('Best Val Acc:', max(history.history['val_accuracy']))  # 최고 검증 정확도 출력
print('Best Val Loss:', min(history.history['val_loss']))  # 최저 검증 손실 출력